In [141]:
import pandas as pd
import numpy as np
from mailer import Mailer
from mailer import Message

In [142]:
#!conda listhttp://localhost:8890/notebooks/Desktop/pathways/J3T_pathway.ipynb#

In [143]:
#!pip install Mailer

In [144]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [145]:
students = pd.read_excel("J3T_admitted.xlsx", sheetname='base')
enrolled_2017 = pd.read_excel("J3T_enrolled_2017.xlsx", sheetname='Sheet1')

In [167]:
fisheries = pd.read_excel("J3T.xlsx", sheetname='fisheries')
aqua = pd.read_excel("J3T.xlsx", sheetname='aquaculture')
conservation = pd.read_excel("J3T.xlsx", sheetname='conservation')

In [147]:
students.head()

,StudentID,FamilyName,FirstGiven Name,StudyPackageCode,StudyPackageCPValue,StudyPackageTitle,PackageStatus,AvailabilityYear,GradeCode,Mark,ParentCode,StudyPackageTitle.1,Email,Location,StudyPeriod,TotalEnrolledCredit,AdvancedStandingCredit,TotalCreditPoints,TotalAchieved
0,191721,Nicholls,Andrew,JFA104,12.5,Fishing Industry and Operations,Passed,2015,DN,75,J3T,BAppSc (ME),Andrew.Nicholls@utas.edu.au,Launceston,Semester 2,62.5,0.0,300,225.0
1,191721,Nicholls,Andrew,JFA219,12.5,Field Research Techniques for Aquatic Ecosystems,Passed,2016,DN,76,J3T,BAppSc (ME),Andrew.Nicholls@utas.edu.au,Launceston,Spring school,62.5,0.0,300,225.0
2,191721,Nicholls,Andrew,JFA213,12.5,Marine Resource Economics,Passed,2016,CR,65,J3T,BAppSc (ME),Andrew.Nicholls@utas.edu.au,Launceston,Semester 1,62.5,0.0,300,225.0
3,191721,Nicholls,Andrew,JFA108,12.5,Introduction to Fisheries Management,Passed,2015,HD,82,J3T,BAppSc (ME),Andrew.Nicholls@utas.edu.au,Launceston,Semester 1,62.5,0.0,300,225.0
4,191721,Nicholls,Andrew,JFA101,12.5,Aquatic Biology A,Passed,2015,DN,71,J3T,BAppSc (ME),Andrew.Nicholls@utas.edu.au,Launceston,Semester 1,62.5,0.0,300,225.0


# Drop all units not passed

In [148]:
fisheriesStudents = students[students['StudyPackageCode'] == 'JFA104']
conservationStudents = students[students['StudyPackageCode'] == 'JFA103']
aquacultreStudents = students[students['StudyPackageCode'] == 'JFA106']

In [149]:
fisheriesStudents['StudentID'].nunique()
conservationStudents['StudentID'].nunique()
aquacultreStudents['StudentID'].nunique()
students['StudentID'].nunique()

31

11

22

70

In [150]:
df2 = students[(students.StudyPackageCode == 'JFA104') | (students.StudyPackageCode == 'JFA106') | (students.StudyPackageCode == 'JFA103')]
df2['StudentID'].nunique()
students.StudentID.nunique()
enrolledList = df2.StudentID.unique()
allStudentsList = students.StudentID.unique()
unenrolledStudents = np.setdiff1d(allStudentsList, enrolledList)
print "Students that are not enrolled in a major: " ,unenrolledStudents

61

70

Students that are not enrolled in a major:  [ 82224 136731 209797 227940 402327 426854 436744 455797 457727]


# Student NOT enrolled in a major

In [151]:
notE = students[students['StudentID'].isin(unenrolledStudents)]

In [152]:
notECP = notE.groupby('StudentID').apply(lambda x: sum(x['StudyPackageCPValue']))
print "CP weights and ", notECP

CP weights and  StudentID
82224      37.5
136731     37.5
209797    150.0
227940    100.0
402327    112.5
426854     87.5
436744     50.0
455797     87.5
457727     50.0
dtype: float64


In [153]:
enrolled_2017.head(1)

,StudentID,FamilyName,FirstName,StudyPackageCode,StudyCPValue,StudyPackageTitle,Status,Year,EmailAddress,Location,StudyPeriod,TotalEnrolledCreditPoints,TotalCreditPoints,TotalAchievedPoints,FieldofEducation
0,191721,Nicholls,Andrew,JFA308,12.5,Fisheries Assessment,Enrolled,2017,Andrew.Nicholls@utas.edu.au,Launceston,Semester 1,62.5,300,225.0,50701


#  Students not enrolled in full Semester 2 load 

In [154]:
sem2 = enrolled_2017[enrolled_2017.StudyPeriod == 'Semester 2']

In [155]:
sem2CP = sem2.groupby(['StudentID','EmailAddress','FamilyName', 'FirstName']).apply(lambda x: sum(x['StudyCPValue']))
notFullLoad = sem2CP[sem2CP < 50]
notFullLoad = pd.DataFrame(notFullLoad)

In [156]:
notFullLoad.to_csv("notFullLoad.csv")

In [157]:
#for index, series in notFullLoad.iterrows():
 #   print index
    #print series[index[0]]
    #message = Message(From = "AMC.courseinfo@utas.edu.au", To = index, charset="utf-8")
    #message.Subject = "J3T Enrolment for Semester 2 2017"
    #message.Html = "Hi, %s! your enrollment score is %s" % (series[0], series[1])
    #message.Html = "Hi, %s! your enrollment score is %s" % (row['FirstName'], row['score'])
   
    
    #sender = Mailer('smtp.gmail.com', port= 465, use_tls=False, usr="linus.mcmanamey@gmail.com", pwd="blockAll3q123", use_ssl=True, use_plain_auth=True)
    #sender.send(message)

In [158]:
aStudents = aquacultreStudents['StudentID'].unique()
aquaCourseList = aqua.course_code
students = students[students['PackageStatus'] == 'Passed']
temp = []
inPathway = []
for i in aStudents:
    temp.append(np.setdiff1d(aquaCourseList, students[students['StudentID']==i]['StudyPackageCode']))
    inPathway.append(np.intersect1d(aquaCourseList, students[students['StudentID']==i]['StudyPackageCode']))
#results = pd.concat(temp)

In [165]:
inPathway

[array([u'JFA101', u'JFA102', u'JFA106', u'JFA107', u'JFA114', u'JFA115',
        u'KMA153', u'KRA101'], dtype=object),
 array([u'JFA101', u'JFA102', u'JFA106', u'JFA107', u'JFA114', u'JFA115',
        u'JFA202', u'JFA203', u'JFA207', u'JFA210', u'JFA214', u'JFA216',
        u'KMA153', u'KRA161'], dtype=object),
 array([], dtype=object),
 array([u'JFA101', u'JFA106', u'JFA107', u'JFA115'], dtype=object),
 array([u'JFA101', u'JFA102', u'JFA106', u'JFA107', u'JFA114', u'JFA115',
        u'KMA153', u'KRA161'], dtype=object),
 array([u'JFA102', u'JFA106', u'JFA107', u'JFA112', u'JFA114', u'JFA115',
        u'JFA202', u'JFA203', u'JFA207', u'JFA302', u'JFA303'], dtype=object),
 array([], dtype=object),
 array([u'JFA101', u'JFA102', u'JFA106', u'JFA107', u'JFA114', u'JFA115',
        u'KMA153', u'KRA161'], dtype=object),
 array([], dtype=object),
 array([u'JFA101', u'JFA102', u'JFA106', u'JFA114', u'JFA115', u'KMA153',
        u'KRA101'], dtype=object),
 array([u'JFA101', u'JFA102', u'JFA106

In [160]:
aquaCourseList

0     JFA101
1     JFA106
2     JFA114
3     KRA101
4     KRA161
5     JFA102
6     JFA107
7     JFA112
8     KMA153
9     JFA115
10    JFA203
11    JFA210
12    JFA214
13    JFA202
14    JFA207
15    JFA216
16    JFA301
17    JFA304
18    JFA305
19    JFA303
20    JFA314
21    JFA302
Name: course_code, dtype: object

In [164]:
matrix = np.matrix([aStudents, temp])
df = pd.DataFrame(data=matrix)
df = df.T
df.columns = ['StudentID', 'units']
df.to_csv("aquapathways.csv", index=False)

IOError: [Errno 13] Permission denied: 'aquapathways.csv'

# fisheriesStudents

In [ ]:
fStudents = fisheriesStudents['StudentID'].unique()
fishCourseList = fisheries.course_code
students = students[students['PackageStatus'] == 'Passed']
temp = []
for i in fStudents:
    temp.append(np.setdiff1d(fishCourseList, students[students['StudentID']==i]['StudyPackageCode']))

In [ ]:
matrix = np.matrix([fStudents, temp])
df = pd.DataFrame(data=matrix)
df = df.T
df.columns = ['StudentID', 'units']
df.to_csv("fishpathways.csv", index=False)

# conservationStudents

In [ ]:
cStudents = conservationStudents['StudentID'].unique()
conservationCourseList = conservation.course_code
students = students[students['PackageStatus'] == 'Passed']
temp = []
for i in cStudents:
    temp.append(np.setdiff1d(conservationCourseList, students[students['StudentID']==i]['StudyPackageCode']))

In [ ]:
matrix = np.matrix([cStudents, temp])
df = pd.DataFrame(data=matrix)
df = df.T
df.columns = ['StudentID', 'units']
df.to_csv("conservpathways.csv", index=False)

In [168]:
fisheries

,discpline,year,sem,course_code,unit_name
0,Core,1,1,JFA101,Aquatic Biology A
1,Core,1,1,JFA114,Introduction to Oceanography
2,major,1,1,JFA108,Intro to Fisheries Management
3,minor,1,1,JFA110,Intro to Marine Conservation
4,Core,1,2,JFA102,Aquatic Biology B
5,major,1,2,JFA104,Fisheries Industry & Operations
6,minor,1,2,JFA107,Intro to Economics
7,Core,1,2,KMA153,Data Handl. & Stats
8,Core,1,2,JFA112,Intro to Quantitative Meth
9,Core,2,1,JFA210,Foundations of Ecology
